# Lesson 2 : LangGraph Components

In [1]:
import os
os.system("clear")
abs_path = os.path.dirname(os.path.abspath(__name__))
os.chdir(abs_path)
print(f"Working directory: <{os.getcwd()}>")

Working directory: </home/tjamil/Desktop/Agents/code_excercises/2.Langgraph/Ai Agents using Langgraph (dl.ai)>


In [2]:
#from dotenv import load_dotenv
#_ = load_dotenv()

In [3]:
from setup_environment_r1 import set_environment_variables
set_environment_variables("LangGraph Basics - deeplearning.ai")

reading environment variables from: </home/tjamil/Desktop/Agents/code_excercises/2.Langgraph/Ai Agents using Langgraph (dl.ai)>
API Keys loaded and tracing set with project name:  LangGraph Basics - deeplearning.ai


In [4]:
#%pip install -U langchain_groq langgraph

In [5]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

#from langchain_openai import ChatOpenAI
import os
from langchain_groq import ChatGroq
model = ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview", api_key=os.environ.get("GROQ_API_KEY"))

/home/tjamil/.local/share/virtualenvs/Agents-4yRd6dJo/lib/python3.11/site-packages/langchain_community/tools/tavily_search/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.tools.tavily_search.tool import (


In [6]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)   # this is the name of the tool taht will be used by LLM

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [7]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [8]:
class Agent:

    def __init__(self, model, tools, system=""):
        #setting up the system message
        self.system = system
        
        #defining the graph
        graph = StateGraph(AgentState)
        
        #adding nodes (llm, action/tools)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        
        #adding conditional edges (decide if action should be taken or not, and which node to go to)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        
        #adding edges
        graph.add_edge("action", "llm")
        
        #setting entry point
        graph.set_entry_point("llm")
        
        #compiling the graph (creates a runnable graph)
        self.graph = graph.compile()
        
        #defining the tools
        self.tools = {t.name: t for t in tools}
        
        #defining the model with tools binding
        self.model = model.bind_tools(tools)

    #defining the function for conditional edge 
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    #defining 'llm node' function
    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    #defining 'action node' function
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls    
        # .too_call is attribute returned aalongside message by functiona calling llms
        
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
                print(f"Result: {result}")

            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        
        print("Back to the model!")
        return {'messages': results}

In [10]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

#model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [11]:
from IPython.display import Image

#Image(abot.graph.get_graph().draw_png())

In [12]:
messages = [HumanMessage(content="What is the weather in sf?. Give just suffiucient information.")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_pvyb', 'type': 'tool_call'}
Result: AttributeError("'FieldInfo' object has no attribute 'results'")
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather forecast for San Francisco'}, 'id': 'call_decn', 'type': 'tool_call'}
Result: AttributeError("'FieldInfo' object has no attribute 'results'")
Back to the model!


In [13]:
result

{'messages': [HumanMessage(content='What is the weather in sf?. Give just suffiucient information.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pvyb', 'function': {'arguments': '{"query": "current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_decn', 'function': {'arguments': '{"query": "weather forecast for San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 301, 'total_tokens': 369, 'completion_time': 0.214783729, 'prompt_time': 0.02229261, 'queue_time': 0.010779118, 'total_time': 0.237076339}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d149e7e0-0f46-4283-85b9-f1b769c67ee9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'cu

In [14]:
result['messages'][-1].content

"I'm sorry, I encountered an error while retrieving the weather information for San Francisco. Could you please provide me with a different query or more details about what you're looking for?"

In [15]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_j2qm', 'type': 'tool_call'}
Result: AttributeError("'FieldInfo' object has no attribute 'results'")
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_5be4', 'type': 'tool_call'}
Result: AttributeError("'FieldInfo' object has no attribute 'results'")
Back to the model!


In [16]:
result['messages'][-1].content

"I'm sorry, I encountered an error while trying to retrieve the weather information for San Francisco and Los Angeles. It seems like there's an issue with the search engine I'm using."

In [18]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

#model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'who won the super bowl in 2024'}, 'id': 'call_22hf', 'type': 'tool_call'}
Result: AttributeError("'FieldInfo' object has no attribute 'results'")
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'headquarters location of the winning team'}, 'id': 'call_q6ww', 'type': 'tool_call'}
Result: AttributeError("'FieldInfo' object has no attribute 'results'")
Calling: {'name': 'tavily_search_results_json', 'args': {'query': "GDP of the state where the winning team's headquarters is located"}, 'id': 'call_fwdd', 'type': 'tool_call'}
Result: AttributeError("'FieldInfo' object has no attribute 'results'")
Back to the model!


In [19]:
print(result['messages'][-1].content)

I'm sorry, but I encountered an error while searching for the information you requested. It seems like there was a problem with the search engine. I'll need to try a different method to find the answers to your questions.
